In [43]:
import os, glob
import cv2, sympy
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols
from sympy.matrices import *
from PIL import Image

In [44]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [45]:
def Convert_to_PNG():
    os.chdir(os.path.join(Root,'Images_Folder'))
    image_patterns = ["*.jpeg", "*.jpg"]
    for pattern in image_patterns:
        for image_file in glob.glob(pattern):
            if not image_file.endswith(".png"):
                new_file = os.path.splitext(image_file)[0] + ".png"
                os.rename(image_file, new_file)
    os.chdir(Root)

In [46]:
def Convert_Image_To_Matrix(Image_Name):
    if Image_Name in os.listdir('Images_Folder'):
        print("Found the picture:", Image_Name)
        Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
        Gray_Image = np.zeros((Image.shape[0], Image.shape[1])) # Tạo ma trận 0 bằng kích thước với ma trận xuất ra từ hình ảnh
        for Row in range(Image.shape[0]): # chạy qua các hàng
            for Col in range(Image.shape[1]): # chạy qua các cột
                Pixel = Image[Row, Col]  # Lấy các giá trị Pixel của định dạng BRG
                Gray_Pixel = Pixel[0] * 0.114 + Pixel[1] * 0.587 + Pixel[2] * 0.299 # Chuyển sang Pixel của ảnh định dạng Gray
                Normalization = Gray_Pixel / 255 # Chuẩn hóa dữ liệu của mỗi Pixel bằng cách chia cho 255
                Gray_Image[Row, Col] = Normalization # Thêm các giá trị vào ảnh Gray
        return Gray_Image
    else: print("Picture not found!!!!")

In [47]:
def Create_Zero_Matrix(Values):
    Zero_Matrix = np.zeros((Values.shape[0], Values.shape[1]))
    return Zero_Matrix

def Create_Identity_Matrix(Matrix):
    Identity_Matrix = Create_Zero_Matrix(Matrix)
    for Row_Column in range(Identity_Matrix.shape[1]):
        Identity_Matrix[Row_Column,Row_Column] = 1
    return Identity_Matrix

In [48]:
def Find_Eigenvalue(Matrix):
    Identity_Matrix = Create_Identity_Matrix(Matrix)
    x = symbols('x') # Tạo ra biến tên là x thay cho λ để tính:  p(λ) = det(A - λI)
    Characteristic_Polynomial = Matrix - x * Identity_Matrix
    return sympy.solve(sympy.det(Characteristic_Polynomial))

def Find_Eigenvector(matrix):
  arr = []
  w,v = np.linalg.eigh(matrix)
  return w,v


In [49]:
def Sort(Eigvaule_Vector):
    dict = {}
    for x in range(len(Eigvaule_Vector[0])):
        dict[Eigvaule_Vector[0][x]] = Eigvaule_Vector[1][:, x]
    key = list(dict.keys())
    key = sorted(key, key=None, reverse=True)
    return {i: dict[i] for i in key}

In [50]:
def Show_Image(Name, Result,k):
    os.chdir(os.path.join(Root, 'Result_Folder'))
    picture_name = Name
    img_array = np.array(Result, dtype=np.uint8)
    img = Image.fromarray(img_array)
    
    if picture_name in os.listdir():
        img = cv2.imread(picture_name)
        
    Filename = f"{k}_SVD Image Compression.jpg"
    
    if not os.path.exists(os.path.join(Root, 'Result_Folder')):
        os.makedirs(os.path.join(Root, 'Result_Folder'))
    
    cv2.imwrite(os.path.join(Root,'Result_Folder', Filename), img_array)

In [51]:
class Calculation:
    def __init__(self, Name):
        self.matrix = Convert_Image_To_Matrix(Name)
        self.shape = np.shape(self.matrix)
        self.k = 5
        result = self.SVD()
        Show_Image(Name, result*255, self.k)


    def SVD(self):
      c,r = self.shape
      A = self.matrix
      V = []
      U = []
      print(np.shape(self.matrix))
      AtA = np.matmul(np.transpose(self.matrix),self.matrix)

      b = Find_Eigenvector(np.array(AtA,dtype= float))

      """turn into dict then soft in descending"""
      Newdict = Sort(b)

    # Tính UDV^T
      D = np.zeros(self.shape,dtype = 'complex_')
      r,c = self.shape
      for x in range(min(r,c)):
          D[x][x] = np.sqrt(np.abs(list(Newdict.keys())[x]))
      d = D[0:self.k,0:self.k]


      for x in Newdict.values():
          V.append(list(x))
      V = Matrix(V).T
      v = V[:,0:self.k]

      for i in range(min(self.k,r)):
        U.append((np.matmul(A,V[:,i])/D[i][i]).T[0])
      U = Matrix(U).T
      u = U[:, 0:self.k]

      a = u * d * (v.T)

      print('D = ',Matrix(d))
      print('V =',v)
      print('U = ',u)
      print('A =', a) 

      return a


if __name__ == '__main__':
    Calculation('Cute Cat.png')


Found the picture: Cute Cat.png
(360, 360)
D =  Matrix([[158.245039593288, 0, 0, 0, 0], [0, 47.3845261260823, 0, 0, 0], [0, 0, 28.5748631399522, 0, 0], [0, 0, 0, 16.2169804774036, 0], [0, 0, 0, 0, 12.2051448685231]])
V = Matrix([[0.0312270530357369, 0.0174082904367911, -0.0286346753263122, 0.123702386106130, -0.0795227046202036], [0.0312911859376708, 0.0178192758778487, -0.0292235746156560, 0.122037224044395, -0.0782367400918419], [0.0312869203513666, 0.0180062690509682, -0.0297099666058547, 0.121133708515318, -0.0764678568144895], [0.0313049093429086, 0.0181170178858896, -0.0298582810052316, 0.120347187147689, -0.0756659162769917], [0.0313171721346788, 0.0183658854611773, -0.0296563850906518, 0.119565040721007, -0.0757486363478791], [0.0313020661049983, 0.0182887858760195, -0.0291898784540630, 0.119195065525930, -0.0761511635127928], [0.0313479559229942, 0.0181419254563062, -0.0285078034699718, 0.119600685405754, -0.0767114651858304], [0.0314546328075092, 0.0182548586686930, -0.028281

In [52]:
os.chdir(Root)
os.getcwd()
# def Show_Image(Name, Result,k):
#     plt.imshow(Image, cmap='gray')
#     plt.axis('off')
#     plt.show() 

'c:\\Users\\QuySech\\OneDrive\\Project\\Singular-Value-Decomposition'